In [3]:
%pip install selenium
%pip install webdriver_manager
%pip install langdetect

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


     -------------------------------------- 981.5/981.5 kB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
Using legacy 'setup.py install' for langdetect, since package 'wheel' is not installed.
  Running setup.py install for langdetect: started
  Running setup.py install for langdetect: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
%pip install pandas

     ---------------------------------------- 11.6/11.6 MB 4.8 MB/s eta 0:00:00
     -------------------------------------- 346.8/346.8 kB 5.4 MB/s eta 0:00:00
     -------------------------------------- 507.9/507.9 kB 6.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import time
import pandas as pd
import re
from langdetect import detect, DetectorFactory

# Pastikan hasil deteksi bahasa konsisten
DetectorFactory.seed = 0

# Setup Selenium untuk Google Chrome
service = Service(ChromeDriverManager().install())
options = Options()
options.add_argument("--start-maximized")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")

# Inisialisasi driver Chrome
driver = webdriver.Chrome(service=service, options=options)

# Query pencarian
query = '"Universitas Brawijaya"'
max_pages = 50  # Batasi jumlah halaman agar tidak diblokir
articles = []

# Fungsi untuk mengecek apakah teks dalam bahasa Inggris
def is_english(text):
    try:
        return detect(text) == "en"
    except:
        return False  # Jika gagal deteksi, anggap bukan bahasa Inggris

# Looping untuk scrape tiap halaman
for page in range(max_pages):
    start = page * 10  # Google Scholar menampilkan 10 artikel per halaman
    search_url = f"https://scholar.google.com/scholar?lr=lang_en&q={query}&start={start}&hl=en"
    
    driver.get(search_url)
    time.sleep(3)  # Tambahkan delay untuk menghindari deteksi bot

    # Periksa apakah ada hasil pencarian
    if "did not match any articles" in driver.page_source:
        print(f"Tidak ada hasil di halaman {page + 1}, berhenti scraping.")
        break

    # Tunggu hingga elemen artikel muncul
    wait = WebDriverWait(driver, 10)
    try:
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "h3.gs_rt")))
    except:
        print(f"Halaman {page + 1} tidak menampilkan artikel, mungkin terblokir.")
        break

    # Ambil semua artikel
    titles = driver.find_elements(By.CSS_SELECTOR, "h3.gs_rt")
    infos = driver.find_elements(By.CSS_SELECTOR, "div.gs_a")  # Info tahun
    abstracts = driver.find_elements(By.CSS_SELECTOR, "div.gs_rs")  # Abstrak

    if not titles:
        print(f"Tidak ada judul ditemukan di halaman {page + 1}, mungkin diblokir.")
        break

    for i in range(len(titles)):
        title_text = titles[i].text.strip() if i < len(titles) else ""
        info_text = infos[i].text if i < len(infos) else ""
        abstract_text = abstracts[i].text.strip() if i < len(abstracts) else ""

        # Ekstrak tahun (4 digit)
        year_match = re.search(r"\b(19|20)\d{2}\b", info_text)
        year = year_match.group(0) if year_match else "Unknown"

        # Simpan hanya jika berbahasa Inggris
        if title_text and is_english(title_text):
            articles.append({"Title": title_text, "Year": year, "Abstract": abstract_text})

    print(f"Halaman {page + 1} selesai, total artikel terkumpul: {len(articles)}")

# Tutup browser
driver.quit()

# Simpan hasil ke dalam file CSV
df = pd.DataFrame(articles)
csv_filename = "universitas_brawijaya_scholar_results2.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"\nHasil scraping telah disimpan ke {csv_filename}")

# Cetak hasil (hanya beberapa contoh)
print("\nBeberapa Judul Artikel yang Ditemukan:")
for i, article in enumerate(articles[:10], 1):
    print(f"{i}. {article['Title']} ({article['Year']})")
    print(f"   Abstract: {article['Abstract'][:200]}...\n")  # Abstrak dipersingkat


Halaman 1 selesai, total artikel terkumpul: 7
Halaman 2 selesai, total artikel terkumpul: 12
Halaman 3 selesai, total artikel terkumpul: 21
Halaman 4 selesai, total artikel terkumpul: 30
Halaman 5 selesai, total artikel terkumpul: 37
Halaman 6 selesai, total artikel terkumpul: 47
Halaman 7 selesai, total artikel terkumpul: 57
Halaman 8 selesai, total artikel terkumpul: 65
Halaman 9 selesai, total artikel terkumpul: 75
Halaman 10 selesai, total artikel terkumpul: 84
Halaman 11 selesai, total artikel terkumpul: 94
Halaman 12 selesai, total artikel terkumpul: 104
Halaman 13 selesai, total artikel terkumpul: 113
Halaman 14 selesai, total artikel terkumpul: 123
Halaman 15 selesai, total artikel terkumpul: 133
Halaman 16 selesai, total artikel terkumpul: 143
Halaman 17 selesai, total artikel terkumpul: 153
Halaman 18 selesai, total artikel terkumpul: 163
Halaman 19 selesai, total artikel terkumpul: 173
Halaman 20 selesai, total artikel terkumpul: 183
Halaman 21 selesai, total artikel terkump

In [2]:
df = pd.read_csv("universitas_brawijaya_scholar_results2.csv")
df.head()

,Title,Year,Abstract
0,[PDF] Management of Public Information Disclos...,Unknown,"Good day, BICMST is the first multidisciplinar..."
1,[PDF] Director Rector of Universitas Brawijaya...,2015,"Keywords: refusal, refusal strategy, gender, c..."
2,Refusal Strategies Used By Male And Female Stu...,2013,… menggambarkan pengembangan Universitas Brawi...
3,Achieving World Class University Through Inter...,2015,"Keywords: Translation, Thesis, Thesis Abstract..."
4,the Translation of Demonstrative References fr...,2023,… pembelajaran pada kalangan mahasiswa Univers...


In [9]:
df['Abstract'].isna()

0      False
1      False
2      False
3      False
4      False
       ...  
466    False
467    False
468    False
469    False
470    False
Name: Abstract, Length: 471, dtype: bool

In [10]:
df.Year

0      Unknown
1         2015
2         2013
3         2015
4         2023
        ...   
466       2013
467       2017
468       2018
469       2019
470       2020
Name: Year, Length: 471, dtype: object

In [15]:
import pandas as pd

# Simpan hasil ke dalam DataFrame
df = pd.DataFrame({"Title": titles_list})

# Simpan ke file CSV
csv_filename = "universitas_brawijaya_scholar_results.csv"
df.to_csv(csv_filename, index=False, encoding="utf-8")

print(f"Hasil scraping telah disimpan ke {csv_filename}")


Hasil scraping telah disimpan ke universitas_brawijaya_scholar_results.csv
